In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 27.5 MB/s 
     |████████████████████████████████| 431 kB 84.8 MB/s 
     |████████████████████████████████| 6.6 MB 75.2 MB/s 
     |████████████████████████████████| 120 kB 76.2 MB/s 
     |████████████████████████████████| 212 kB 89.4 MB/s 
     |████████████████████████████████| 115 kB 77.9 MB/s 
     |████████████████████████████████| 127 kB 81.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
num_labels = 13
model_ckpt = "bert-base-chinese"
device = torch.device("cuda" if torch.cuda. is_available() else "cpu")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=num_labels).to(device)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nlp-guild/intent-recognition-biomedical")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/nlp-guild___csv/nlp-guild--intent-recognition-biomedical-6e3512ca70e4392b/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.shuffle()
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_class', 'label'],
        num_rows: 7273
    })
    test: Dataset({
        features: ['text', 'label_class', 'label'],
        num_rows: 809
    })
})

In [ ]:
label_set = [
'定义',
'病因',
'预防',
'临床表现(病症表现)',
'相关病症',
'治疗方法',
'所属科室',
'传染性',
'治愈率',
'禁忌',
'化验/体检方案',
'治疗时间',
'其他'
]

In [ ]:
dataset['train'][:3]

{'text': ['怎么瘦肚子？', '小孩吃东西就说肚子痛还吐是怎么回事', '妊娠合并重症肌无力的症状表现是什么'],
 'label_class': ['其他', '病因', '临床表现(病症表现)'],
 'label': [12, 1, 3]}

In [ ]:
dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label_class': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [ ]:
def preprocess(sample):
    input = tokenizer(sample['text'],truncation=True)
    input['label'] = sample['label']
    return input

In [ ]:
tokenized_datasets = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset['train'].column_names
)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7273
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 809
    })
})

In [ ]:
from datasets import ClassLabel
feat_sentiment = ClassLabel(num_classes = len(label_set), names=label_set)
tokenized_datasets = tokenized_datasets.cast_column("label", feat_sentiment)

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets['train'].features

{'label': ClassLabel(num_classes=13, names=['定义', '病因', '预防', '临床表现(病症表现)', '相关病症', '治疗方法', '所属科室', '传染性', '治愈率', '禁忌', '化验/体检方案', '治疗时间', '其他'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
tokenized_datasets['train'][:10]['label']

[12, 1, 3, 12, 1, 5, 12, 5, 11, 12]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from datasets import load_metric

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import numpy as np
def compute_metrics(pred):
    logits, labels = pred
    '''
    logits: N,L,D
    labels: N,L
    '''
    predictions = np.argmax(logits, axis=-1) # N,L
    f1 = f1_score(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    recall = recall_score(labels,predictions,average='weighted')
    precision = precision_score(labels,predictions,average='weighted')
    return {"accuracy": acc, "f1": f1, 'recall': recall, 'precision': precision}

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=100,
    save_total_limit=3,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    weight_decay=0.01,
    output_dir=f'{model_ckpt}-finetuned-intent_recognition-biomedical'
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7273
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 91000


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,0.870600,1.164791,0.625464,0.620611,0.625464,0.648184
2,0.770600,1.152896,0.634116,0.627510,0.634116,0.650984
3,0.624100,1.300014,0.620519,0.609619,0.620519,0.655458
4,0.572100,1.242000,0.669963,0.669180,0.669963,0.677787
5,0.553000,1.483133,0.636588,0.623582,0.636588,0.640161
6,0.395300,1.651743,0.656366,0.652905,0.656366,0.666846
7,0.364200,1.821525,0.650185,0.646863,0.650185,0.652653
8,0.350900,1.846293,0.663782,0.663204,0.663782,0.668802
9,0.331500,2.154310,0.645241,0.648263,0.645241,0.659122
10,0.333000,2.259519,0.668727,0.666989,0.668727,0.670927


***** Running Evaluation *****
  Num examples = 809
  Batch size = 8
Saving model checkpoint to bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-910
Configuration saved in bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-910/config.json
Model weights saved in bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-910/pytorch_model.bin
tokenizer config file saved in bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-910/tokenizer_config.json
Special tokens file saved in bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-910/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 809
  Batch size = 8
Saving model checkpoint to bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-1820
Configuration saved in bert-base-chinese-finetuned-intent_recognition-biomedical/checkpoint-1820/config.json
Model weights saved in bert-base-chinese-finetuned-intent_recognition-biomedic

TrainOutput(global_step=91000, training_loss=0.13070225061689103, metrics={'train_runtime': 10772.3299, 'train_samples_per_second': 67.516, 'train_steps_per_second': 8.448, 'total_flos': 1.810151983248367e+16, 'train_loss': 0.13070225061689103, 'epoch': 100.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 809
  Batch size = 8


{'eval_loss': 2.240311861038208,
 'eval_accuracy': 0.6847960444993819,
 'eval_f1': 0.6831087743598181,
 'eval_recall': 0.6847960444993819,
 'eval_precision': 0.6848333597632937,
 'eval_runtime': 2.6293,
 'eval_samples_per_second': 307.686,
 'eval_steps_per_second': 38.794,
 'epoch': 100.0}

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets['train'])

***** Running Evaluation *****
  Num examples = 7273
  Batch size = 8


{'eval_loss': 0.09228955209255219,
 'eval_accuracy': 0.9769008662175168,
 'eval_f1': 0.9768637750603645,
 'eval_recall': 0.9769008662175168,
 'eval_precision': 0.9769398404350202,
 'eval_runtime': 24.9784,
 'eval_samples_per_second': 291.171,
 'eval_steps_per_second': 36.431,
 'epoch': 100.0}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
model.push_to_hub('nlp-guild/bert-base-chinese-finetuned-intent_recognition-biomedical')

Configuration saved in bert-base-chinese-finetuned-intent_recognition-biomedical/config.json
Model weights saved in bert-base-chinese-finetuned-intent_recognition-biomedical/pytorch_model.bin
Uploading the following files to nlp-guild/bert-base-chinese-finetuned-intent_recognition-biomedical: config.json,pytorch_model.bin


In [ ]:
tokenizer.push_to_hub('nlp-guild/bert-base-chinese-finetuned-intent_recognition-biomedical')

tokenizer config file saved in bert-base-chinese-finetuned-intent_recognition-biomedical/tokenizer_config.json
Special tokens file saved in bert-base-chinese-finetuned-intent_recognition-biomedical/special_tokens_map.json
Uploading the following files to nlp-guild/bert-base-chinese-finetuned-intent_recognition-biomedical: special_tokens_map.json,tokenizer.json,tokenizer_config.json,vocab.txt
